<h1> Repeatable splitting </h1>

In this notebook, we will explore the impact of different ways of creating machine learning datasets.

<p>

Repeatability is important in machine learning. If you do the same thing now and 5 minutes from now and get different answers, then it makes experimentation difficult. In other words, you will find it difficult to gauge whether a change you made has resulted in an improvement or not.

In [1]:
pip install --upgrade google-cloud-bigquery[bqstorage,pandas]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 2.34.3
    Uninstalling google-cloud-bigquery-2.34.3:
      Successfully uninstalled google-cloud-bigquery-2.34.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.13.0 requires google-cloud-bigquery<3.0.0dev,>=1.15.0, but you have google-cloud-bigquery 3.2.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import bigquery

<h3> Create a simple machine learning model </h3>

The dataset that we will use is a BigQuery dataset of airline arrival data.
<p>
We want to predict the arrival delay of an airline based on the departure delay. The model that we will use is a zero-bias linear model:
$$ delay_{arrival} = \alpha * delay_{departure} $$
<p>
To train the model is to estimate a good value for $\alpha$. 
<p>
One approach to estimate alpha is to use this formula:
$$ \alpha = \frac{\sum delay_{departure} delay_{arrival} }{  \sum delay_{departure}^2 } $$
Because we'd like to capture the idea that this relationship is different for flights from New York to Los Angeles vs. flights from Austin to Indianapolis (shorter flight, less busy airports), we'd compute a different $alpha$ for each airport-pair.  For simplicity, we'll do this model only for flights between Denver and Los Angeles.

<h2> Naive random split (not repeatable) </h2>

In [3]:
compute_alpha = """
#standardSQL
SELECT 
   SAFE_DIVIDE(SUM(arrdelay * depdelay), SUM(depdelay * depdelay)) AS alpha
FROM
(
   SELECT RAND() AS splitfield,
   arrdelay,
   depdelay
FROM
  `cloud-training-demos.airline_ontime_data.flights`
WHERE
  Origin = 'DEN' AND Dest = 'LAX'
)
WHERE
  splitfield < 0.8
"""

In [4]:
results = bigquery.Client().query(compute_alpha).to_dataframe()
alpha = results['alpha'][0]
print(alpha)

0.9803190466122611


<h3> What is wrong with calculating RMSE on the training and test data as follows? </h3>

In [10]:
compute_rmse = """
#standardSQL
SELECT
  dataset,
  SQRT(AVG((arrdelay - ALPHA * depdelay)*(arrdelay - ALPHA * depdelay))) AS rmse,
  COUNT(arrdelay) AS num_flights
FROM (
  SELECT
    IF (RAND() < 0.8, 'train', 'eval') AS dataset,
    arrdelay,
    depdelay
  FROM
    `cloud-training-demos.airline_ontime_data.flights`
  WHERE
    Origin = 'DEN'
    AND Dest = 'LAX' )
GROUP BY
  dataset
"""
bigquery.Client().query(compute_rmse.replace('ALPHA', str(alpha))).to_dataframe()

,dataset,rmse,num_flights
0,train,13.471226,159762
1,eval,13.339180,40097


Hint:
* Are you really getting the same training data in the compute_rmse query as in the compute_alpha query?
* Do you get the same answers each time you rerun the compute_alpha and compute_rmse blocks?

<h3> How do we correctly train and evaluate? </h3>
<br/>
Here's the right way to compute the RMSE using the actual training and held-out (evaluation) data. Note how much harder this feels.

Although the calculations are now correct, the experiment is still not repeatable.

Try running it several times; do you get the same answer?

In [11]:
train_and_eval_rand = """
#standardSQL
WITH
  alldata AS (
  SELECT
    IF (RAND() < 0.8,
      'train',
      'eval') AS dataset,
    arrdelay,
    depdelay
  FROM
    `cloud-training-demos.airline_ontime_data.flights`
  WHERE
    Origin = 'DEN'
    AND Dest = 'LAX' ),
  training AS (
  SELECT
    SAFE_DIVIDE( SUM(arrdelay * depdelay) , SUM(depdelay * depdelay)) AS alpha
  FROM
    alldata
  WHERE
    dataset = 'train' )
SELECT
  MAX(alpha) AS alpha,
  dataset,
  SQRT(AVG((arrdelay - alpha * depdelay)*(arrdelay - alpha * depdelay))) AS rmse,
  COUNT(arrdelay) AS num_flights
FROM
  alldata,
  training
GROUP BY
  dataset
"""

In [12]:
bigquery.Client().query(train_and_eval_rand).to_dataframe()

,alpha,dataset,rmse,num_flights
0,0.981085,eval,13.406492,39907
1,0.981085,train,13.453973,159952


<h2> Using HASH of date to split the data </h2>

Let's split by date and train.

In [13]:
compute_alpha = """
#standardSQL
SELECT 
   SAFE_DIVIDE(SUM(arrdelay * depdelay), SUM(depdelay * depdelay)) AS alpha
FROM
  `cloud-training-demos.airline_ontime_data.flights`
WHERE
  Origin = 'DEN' AND Dest = 'LAX'
  AND ABS(MOD(FARM_FINGERPRINT(CAST(CURRENT_DATE() AS STRING)), 10)) < 8
"""
results = bigquery.Client().query(compute_alpha).to_dataframe()
alpha = results['alpha'][0]
print(alpha)

0.9795488325223913


We can now use the alpha to compute RMSE. Because the alpha value is repeatable, we don't need to worry that the alpha in the compute_rmse will be different from the alpha computed in the compute_alpha.

In [14]:
compute_rmse = """
#standardSQL
SELECT
  IF(ABS(MOD(FARM_FINGERPRINT(CAST(CURRENT_DATE() AS STRING)), 10)) < 8, 'train', 'eval') AS dataset,
  SQRT(AVG((arrdelay - ALPHA * depdelay)*(arrdelay - ALPHA * depdelay))) AS rmse,
  COUNT(arrdelay) AS num_flights
FROM
    `cloud-training-demos.airline_ontime_data.flights`
WHERE
    Origin = 'DEN'
    AND Dest = 'LAX'
GROUP BY
  dataset
"""
print(bigquery.Client().query(compute_rmse.replace('ALPHA', str(alpha))).to_dataframe().head())

  dataset       rmse  num_flights
0   train  13.444838       199859


Note also that the RMSE on the evaluation dataset more from the RMSE on the training dataset when we do the split correctly.  This should be expected; in the RAND() case, there was leakage between training and evaluation datasets, because there is high correlation between flights on the same day.
<p>
This is one of the biggest dangers with doing machine learning splits the wrong way -- <b> you will develop a false sense of confidence in how good your model is! </b>

Copyright 2018 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.